In [1]:
import os
import random
import pickle
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)
torch.backends.cudnn.deterministic = True

In [58]:
with open('../pickles_for_git/main_dict.pickle', 'rb') as f:
    main_dict = pickle.load(f)

In [6]:
i = 0
req_list = list()
for idx in main_dict:
    if len(main_dict[idx]['all_tags']) < 4:
        req_list.append(idx)


In [11]:
len(req_list)

2024

In [12]:
import pylast
from tqdm import tqdm
import time

API_KEY = "80113e47bfd1c58983acc18b4d46e025"
API_SECRET = "604648c3611f684b077c83df8e3a00fd"
username = "obi-1-konobe"
password_hash = pylast.md5("T4s4rt!rlastfm")

In [10]:
network = pylast.LastFMNetwork(
            api_key=API_KEY,
            api_secret=API_SECRET,
            username=username,
            password_hash=password_hash
        )

In [69]:
def get_response_lastfm(tracks_list):
    result_dict = dict()
    errors = list()
    for idx in tqdm(tracks_list):
        artist_name = main_dict[idx]['artist_name']
        track_title = main_dict[idx]['track_title']

        try:
            artist = network.get_artist(artist_name)
#             track = network.get_track(artist, track_title)

            artist_tags = artist.get_top_tags(10)
#             track_tags = track.get_top_tags(10)
#             similar_track = track.get_similar()
        except:
            errors.append(idx)
        else:
            temp_dict = dict()
            temp_dict['artist_name'] = artist_name
            temp_dict['track_title'] = track_title
            temp_dict['artist_tags'] = [(elem.item.name, elem.weight) for elem in artist_tags]
#             temp_dict['track_tags'] = [(elem.item.name, elem.weight) for elem in track_tags]
#             temp_dict['similar_tracks'] = [(elem.item.artist.name, elem.item.title, elem.match) for elem in similar_track]

            result_dict[idx] = temp_dict
        time.sleep(0.5)

    result_dict['errors'] = errors
    return result_dict

In [67]:
artist = network.get_artist('Solemn Man')
track = network.get_track(artist, 'Track 07')

artist_tags = artist.get_top_tags(10)
track_tags = track.get_top_tags(10)
similar_track = track.get_similar()

WSError: Track not found

In [15]:
req_dict = get_response_lastfm(req_list)

100%|██████████| 2024/2024 [1:00:41<00:00,  1.80s/it]  


In [18]:
len(req_dict['errors'])

539

In [70]:
req_dict_3 = get_response_lastfm(req_dict_2['errors'])

100%|██████████| 536/536 [07:54<00:00,  1.13it/s]


In [86]:
main_dict[1685]

{'artist_name': 'The Sounds of Taraab',
 'track_title': 'Eni Wana Damu',
 'artist_tags': [],
 'track_tags': [],
 'similar_tracks': [],
 'similar_list': [],
 'genre': 'International',
 'all_genres': '[2]',
 'fma_tags': '[]',
 'all_tags': ['oriental',
  'ethno',
  'african',
  'ethno music',
  'world music',
  'crossover',
  'taarab',
  'recent music heroes',
  'ethno pop',
  'international',
  'islamic']}

In [79]:
for idx in req_dict_3:
    a = req_dict_3[idx]['artist_tags']
#     b = req_dict[idx]['track_tags']
    a_tags = [x[0].lower() for x in a]
#     b_tags = [x[0].lower() for x in b]
    all_tags = a_tags
    req_dict_3[idx]['all_tags'] = all_tags

In [81]:
for idx in req_dict_3:
    a = main_dict[idx]['all_tags']
    b = req_dict_3[idx]['all_tags']
    c = list(set(a + b))
    main_dict[idx]['all_tags'] = c

In [84]:
i = 0
# req_list = list()
for idx in main_dict:
    if len(main_dict[idx]['all_tags']) < 2:
#         req_list.append(idx)
        i+=1
i


478

In [87]:
with open('../pickles_for_git/main_dict.pickle', 'wb') as f:
    pickle.dump(main_dict, f)